# Semantic Product Search Benchmarks

In the jupyterconsole download the following packages:
pip install datasets, accelerate, transformers[torch], rank_bm25

In [1]:
import os
import math
import pandas as pd
import networkx as nx
import numpy as np
import torch.nn.functional as F
import time
import torch
import pickle
import random
import tqdm
import math
from sklearn.preprocessing import LabelEncoder
from torch_geometric.utils import from_networkx
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
from torch_geometric.nn import GCNConv, FastRGCNConv, RGCNConv, RGATConv
from sklearn.metrics import average_precision_score
from sklearn.metrics.pairwise import cosine_similarity
from torch.utils.data import DataLoader, random_split
from sentence_transformers import InputExample
from sentence_transformers import losses
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CERerankingEvaluator
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers import evaluation
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sentence_transformers import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CrossEncoderRerankingEvaluator

from sklearn.metrics import average_precision_score, ndcg_score
import re
from sentence_transformers import CrossEncoder, InputExample, losses, models
from torch.utils.data import DataLoader
import logging
import traceback
from torch.nn import MSELoss
from sentence_transformers import CrossEncoder
from sentence_transformers.cross_encoder import CrossEncoderTrainer, CrossEncoderTrainingArguments
#from datasets import Dataset, Features, Value
from sentence_transformers.cross_encoder import (
    CrossEncoder,
    CrossEncoderModelCardData,
    CrossEncoderTrainer,
    CrossEncoderTrainingArguments,
)
from sentence_transformers.cross_encoder.evaluation import CrossEncoderNanoBEIREvaluator
from sentence_transformers.cross_encoder.losses import CachedMultipleNegativesRankingLoss

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sentence_transformer_model = SentenceTransformer('all-MiniLM-L6-v2')

## Select Subset, Trainingversio and dataset

In [3]:
#amazon = "wands"
amazon = "amazon"

training_version = "shuffle"
#training_version = "predict"

test_subset = 0 # 0...5

### Import Data

In [4]:
if amazon == "amazon":
    df_examples = pd.read_parquet('data/esci-data/shopping_queries_dataset_examples.parquet')
    df_products = pd.read_parquet('data/esci-data/shopping_queries_dataset_products.parquet')
    
    df_examples = df_examples[df_examples["product_locale"] == "us"]
    df_products = df_products[df_products["product_locale"] == "us"]
    
    # Reduce to smaller subset
    df_examples = df_examples[df_examples["small_version"] == 1]

else:
    df_products = pd.read_csv("data/wands-data/product.csv", sep='\t')
    query_df = pd.read_csv("data/wands-data/query.csv", sep='\t')
    label_df = pd.read_csv("data/wands-data/label.csv", sep='\t')
    df_examples = label_df

# we want to split data early, reduce unnecessary data cleaning,...
match test_subset:
    case 0:
        desired_length = 10000
        rkey = 42
    case 1:
        desired_length = 10000
        rkey = 43
    case 2:
        desired_length = 10000
        rkey = 69
    case 3:
        desired_length = 50000
        rkey = 42
    case 4:
        desired_length = 50000
        rkey = 43
    case 5:
        desired_length = 50000
        rkey = 69
    case 6:
        desired_length = 300000
        rkey = 369

rng = np.random.default_rng(rkey)
    
qids = df_examples['query_id'].unique() # all querys
num_queries = int(desired_length/(len(df_examples)/ len(qids))) # desired length/ avg depth
# for calculating ca. desired length many queries
qids_to_use = rng.choice(qids, size=num_queries, replace=False)

# include all querys from those qids and only the included products
df_examples = df_examples[df_examples["query_id"].isin(qids_to_use)]
df_products = df_products[df_products["product_id"].isin(df_examples["product_id"])]

## Save the qids in csv, for retraining
if not os.path.exists(f"data/subset-csvs/{amazon,test_subset}.txt"):
    with open(f'data/subset-csvs/{amazon,test_subset}.txt', 'w') as f:
        f.write(str(qids_to_use))


### Creating some helper functions/ functionality

In [5]:
def clean_color(df_products):
    colors = ["WHITE", "YELLOW", "BLUE", "RED", "GREEN", "BLACK", "BROWN", "AZURE", "IVORY", "TEAL", "SILVER", "PURPLE", "NAVY BLUE", "PEA GREEN", "GRAY", "ORANGE", "MAROON", "CHARCOAL", "AQUAMARINE", "CORAL", "FUCHSIA", "WHEAT", "LIME", "CRIMSON", "KHAKI", "HOT PINK", "MAGENTA", "OLDEN", "PLUM", "OLIVE", "CYAN"]
    pattern = '|'.join(colors)
    
    def process_colors(text):
        if text is None: return '[]'
        matches = re.findall(pattern, text.upper())
        return str(matches) if matches else '[]'
    
    df_products['product_color'] = df_products['product_color'].apply(process_colors)
    return df_products

def clean_prod_desc(df_products):
    def process_info(text):
        if text is None: return text
        r = 0
        text_c = ""
        for i in text:
            if r == 0 and i != "<": text_c += i
            elif i == ">":
                r = 0
            else: 
                r = 1
        return text_c
    df_products['product_description'] = df_products['product_description'].apply(process_info)
    return df_products

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           "]+", flags=re.UNICODE)

def clean_emojis_symbols(text):
    return text.astype(str).apply(lambda x: emoji_pattern.sub("", x))

def create_label_dict(df, label, label_map):
    return {(row['query_id'], row['product_id']): label_map[row[label]] for _, row in df.iterrows()}

In [6]:
if amazon == 'amazon':
    label = 'esci_label'
    product_txt = 'product_title'
    df_products = clean_prod_desc(df_products)
    df_products = clean_color(df_products)
    df_products['product_bullet_point'] = clean_emojis_symbols(df_products['product_bullet_point'])
    df_products['product_title'] = clean_emojis_symbols(df_products['product_title'])
    product_infos = ['product_title', 'product_description', 'product_bullet_point'] 
else:
    label = 'label' 
    product_txt = 'product_name'
    df_products['product_name'] = clean_emojis_symbols(df_products['product_name'])
    df_products['product_features'] = clean_emojis_symbols(df_products['product_features'])
    df_examples = pd.merge(df_examples, query_df, how='left', on='query_id')
    product_infos = ['product_name', 'product_description', 'product_features']

# Sorting by query-id, so that train, val, test splits are useful
df_examples = df_examples.sort_values(by='query_id')

# Assign a unique sequential node ID to each product (for easier accessing) from 0 to len(df_products)
df_products = df_products.reset_index(drop=True).copy()
df_products['node_id'] = range(len(df_products))
# Map between original product indices and node_ids
product_id_to_node_id = dict(zip(df_products['product_id'], df_products['node_id']))
node_id_to_product_id = dict(zip(df_products['node_id'], df_products['product_id']))

# Create a lookup table for product relevancy
label_map = {"E": 1, "S": 0.1, "C":0.01, "I": 0, "Exact":1, "Partial":0.2, "Irrelevant":0}
label_dict = create_label_dict(df_examples, label, label_map)

all_product_info = []
for _,row in df_products.iterrows():
    all_product_info.append(" ".join(str(row[field]) for field in product_infos if pd.notnull(row[field])))

## Create Train, Eval, Test Data

In [7]:
total_len = len(df_examples)
train_parts = int(0.7*total_len)
eval_parts = int(0.1*total_len)
test_parts = total_len - eval_parts - train_parts

In [8]:
if training_version == 'predict' and amazon == 'amazon': # to predict unseen questions
    train_data = df_examples[df_examples['split']== 'train'] 
    train_data, eval_data = train_data[:int(len(train_data)*.8)],train_data[int(len(train_data)*0.8+1):]
    test_data = df_examples[df_examples['split']== 'test'] 
elif training_version == 'predict':
    train_data = df_examples.iloc[:train_parts,:]
    eval_data = df_examples.iloc[train_parts:(train_parts+eval_parts),:]
    test_data = df_examples.iloc[(train_parts+eval_parts):,:]
else:
    splits = random_split(
        df_examples,
        [train_parts, eval_parts, test_parts],
        generator=torch.Generator().manual_seed(42)
    )
    train_idx, val_idx, test_idx = [list(s.indices) for s in splits]
    
    train_data = df_examples.iloc[train_idx]
    eval_data  = df_examples.iloc[val_idx]
    test_data  = df_examples.iloc[test_idx]
    
old_test_data = test_data.copy()
test_data = test_data[['query','query_id']].drop_duplicates()

In [9]:
print("Größe Subset Examples:",len(df_examples))
print("Größe Subset Products:",len(df_products))
print("Amount of queries:",len(df_examples['query_id'].unique()))
print("Größe Subset Training:", len(train_data))
print("Größe Eval Training:", len(eval_data))
print("Größe Subset Test:", len(test_data))
print("AVG DEPTH:",len(df_examples)/len(df_examples['query_id'].unique()))
depth = len(df_examples)/len(df_examples['query_id'].unique())
counti = {}
for value in df_examples[label]:
    if value not in counti:
        counti[value] = 0
    counti[value] += 1
print(counti) 
#with open(f'outputs/subset-specifics.txt', 'a') as f:
 #   f.write(f'{amazon}{test_subset} :: {len(df_examples)}, {len(df_products)} ,{depth} , {counti} \n\n')

Größe Subset Examples: 9755
Größe Subset Products: 9671
Amount of queries: 496
Größe Subset Training: 6828
Größe Eval Training: 975
Größe Subset Test: 487
AVG DEPTH: 19.66733870967742
{'S': 3548, 'I': 1631, 'C': 369, 'E': 4207}


In [11]:
class QueryDataset(Dataset):
    def __init__(self, df, batch_size=64):
        self.pairs = []
        query_ids = df['query_id'].tolist()          # query, qid, product_id
        queries = df['query'].tolist()
        
        all_embeddings = [query.split(" ") for query in queries]

        for emb, qid in zip(all_embeddings, query_ids):
            self.pairs.append((emb, qid))


    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        emb, qid = self.pairs[idx]
        return emb, qid
        
def collate_fn(batch):
    emb = [item[0] for item in batch]
    query_ids = [item[1] for item in batch]
    return emb, query_ids

dataset = QueryDataset(test_data)
test_dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [12]:
class Evaluator:
    def __init__(self, dataloader, product_emb):
        self._dataloader = dataloader
        self._product_embeddings = product_emb # doesnt need to be emb, can also be just strings
    
        """ 
    def forwards(self, query):
        # needs to get implemented by every one calling eval
        """
    
    def evaluate(self, skip=False):
        self.skip = skip
        # called self.forward
        if self.skip: mrr, ndcg, p_at_k, r_precision, sci_ap = [], [], [], [], 0
        else: mrr, ndcg, p_at_k, r_precision, sci_ap= [], [], [], [], []
            
        
        with torch.no_grad():
            for queries, qids in self._dataloader:
                for i in range(len(queries)):
                    #sim_scores = bm25.get_scores(queries[i])
                    sim_scores = self.forwards(queries[i], self._product_embeddings)
                    
                    # Represents the node-indexes with decreasing similarity
                    sim_indx = np.argsort(sim_scores)[::-1].tolist()
                   
                    
                     # Create relevance vector            
                    relevance = [0]*len(sim_indx)
                    for j in range(len(sim_indx)):
                        pid = node_id_to_product_id[sim_indx[j]]
                        if type(qids[i]) != int:
                            rel = label_dict.get((qids[i].item(),pid), 0)
                        else:
                            rel = label_dict.get((qids[i],pid), 0)
                        if rel < 1: continue
                        relevance[j] = 1
        
                    # Total of positive matches
                    total_pos = sum(relevance)
                    
                    if total_pos == 0: 
                        continue
                        
                    relevance = np.array(relevance)
        
                    # Precision at k= relevant geschnit retrieved / retrieved
                    def p_for_k(x): 
                        return sum(relevance[:x+1]) / (x+1)
        
                    # Precision at top k
                    p_at_k.append(p_for_k(10))
        
                    # R-Precision: relevant, among sum relevant
                    r_precision.append(sum(relevance[:total_pos])/total_pos)
              
                    # MRR 
                    ranks = np.where(relevance == 1)[0]
                    if len(ranks) > 0:
                        mrr.append(1.0 / (ranks[0] + 1))
                    else:
                        mrr.append(0)
        
                    # NDCG
                    def dcg():
                        dcg_im = 0
                        for l in ranks.tolist():
                            dcg_im += 1 /( np.log2(l + 1 +1)) # bcs we start at 0
                        return dcg_im
                        
                    def idcg(): 
                        idcg_im = 0
                        for n in range(total_pos):
                            idcg_im += 1 / (np.log2(n + 1 + 1))
                        return idcg_im
                    ndcg.append(dcg() / idcg())
        
                    # MAP with Scikit-learn ony if skip == False
                    if not self.skip:
                        def maper(x): 
                            if x == '1': 
                                return 1 
                            else: 
                                return 0 
                        y_true = np.array([maper(str(label_dict.get((qids[i],c), 0)))   for c in df_products['product_id']])
                        sci_ap.append(average_precision_score(y_true, sim_scores))
                
            return {
                    "MRR": float(np.mean(mrr)),
                    "nDCG": float(np.mean(ndcg)),
                    "R-Precision": float(np.mean(r_precision)),
                    "Precision@k": float(np.mean(p_at_k)),
                    "Scikit-MAP": float(np.mean(sci_ap))
                }

## Database with random guesses

In [13]:
query_to_prod = {}
for _,p in train_data.iterrows():
    query = str(p['query']).strip().lower()
    q = " ".join(query.split())
    if q not in query_to_prod:
        query_to_prod[q] = []
    if p[label] == "Exact" or p[label] == 'E':
        query_to_prod[q].append(p['product_id'])

for l in query_to_prod:
    if query_to_prod[l] is None:
        print("xd")

In [14]:
from random import shuffle

class Eval_db(Evaluator):
    def forwards(self, query, p_embeddings):
        query= " ".join(i for i in query)
        q = query.strip().lower()
        q = " ".join(q.split())
        scores = [0]*len(df_products)
        if q in query_to_prod:
            results = query_to_prod[q]
            for i in results:
                scores[product_id_to_node_id[i]] = 1 
        else:
            print("o")
        for l in range(len(scores)):
            if scores[l] != 1:
                scores[l] = random.uniform(0, 0.9)
        print(scores)
        raise EOFError
        return scores

x = Eval_db(test_dataloader, None)
metrics = x.evaluate()
print(metrics)

{'MRR': 0.9651009326698826, 'nDCG': 0.8231041799566408, 'R-Precision': 0.6888552799937497, 'Precision@k': 0.48665297741273095, 'Scikit-MAP': 0.6885312428138274}


## BM25

In [15]:
from rank_bm25 import BM25Okapi
tokenized_corpus = tokenized_corpus = [prod.split(" ") for prod in all_product_info]
bm25 = BM25Okapi(tokenized_corpus)

In [16]:
class Eval_bm25(Evaluator):
    def forwards(self, query, p_embeddings):
        scores = p_embeddings.get_scores(query)
        return scores

x = Eval_bm25(test_dataloader, bm25)
metrics = x.evaluate()
print(metrics)

{'MRR': 0.37033954224674864, 'nDCG': 0.4027936796923701, 'R-Precision': 0.1835023391199169, 'Precision@k': 0.16483106216165763, 'Scikit-MAP': 0.17278011363155316}


In [17]:
print(metrics)
with open(f'outputs/bm25-{amazon}.txt', 'a') as f:
    f.write(f'bm25-{test_subset,training_version} :: {str(metrics)} \n\n')

{'MRR': 0.37033954224674864, 'nDCG': 0.4027936796923701, 'R-Precision': 0.1835023391199169, 'Precision@k': 0.16483106216165763, 'Scikit-MAP': 0.17278011363155316}


In [40]:
## Manual Query
def man_query(q, top_k = 10):
    tokenized_query = q.split(" ")
    print("Query-Results for:", q)
    print(bm25.get_top_n(tokenized_query, df_products[product_txt], n=top_k))
    
man_query("I want to find hiking shoes for women")

Query-Results for: I want to find hiking shoes for women
['Timberland mens White Ledge Mid Waterproof Hiking Shoe, Wheat, 9.5 US', "Crocs Unisex Men's and Women's Classic Clog, Khaki, 13 US", 'Lined Refillable Vintage Writing Journal for Women, Retro Tree of Life Faux Leather Cover Notebook/Travel Diary,Wide Ruled Paper,Daily Use Gift for Bloggers/Teachers/Back to College Students (Brown)', "Crocs Men's and Women's Classic Tie Dye Clog, Multi, 6 US", 'Thinvik Bike Cleats for Look Keo Grip,Road Bike Look Keo Cleat with Anti-Slip Rubber Desgin.Bicycle Cleat Set for Indoor&Outdoor Cycling- 0 Float Degree', "Timberland Women's Nellie Double Waterproof Ankle Boot,Navy,8 M US", "Timberland Women's Nellie Double Waterproof Ankle Boot,Navy,7.5 M US", "Timberland Women's Nellie Double Waterproof Ankle Boot,Black,7.5 M US Toddler", 'AINIC Pleated Tennis Skirts for Women with Pockets Shorts High Waisted Golf Skorts Athletic Running Workout Sports Skirt White', 'Dirty Sexy Player (Dirty Games Book

## TF-IDF Vectorizer

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tf_embeddings = vectorizer.fit_transform(all_product_info)

In [20]:
class QueryDataset(Dataset):
    def __init__(self, df, batch_size=64):
        self.pairs = []
        query_ids = df['query_id'].tolist()         
        queries = df['query'].tolist()
        all_embeddings = vectorizer.transform(queries)
        
        for emb, qid in zip(all_embeddings, query_ids):
            self.pairs.append((emb, qid))


    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        emb, qid = self.pairs[idx]
        return emb, torch.tensor(qid)

def collate_tfidf(batch):
    emb = [item[0] for item in batch]
    query_ids = torch.stack([item[1] for item in batch])
    return emb, query_ids

In [21]:
dataset = QueryDataset(test_data)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=collate_tfidf)

In [22]:
class Eval_tfidf(Evaluator):
    def forwards(self, query, p_embeddings):
        scores = cosine_similarity(query,p_embeddings)
        return scores[0]

x = Eval_tfidf(dataloader, tf_embeddings)
metrics = x.evaluate(True)   

In [23]:
print(metrics)
with open(f'outputs/tfidf-{amazon}.txt', 'a') as f:
    f.write(f'tfidf-{test_subset,training_version} :: {str(metrics)} \n\n')

{'MRR': 0.5908344854120229, 'nDCG': 0.6165674369477443, 'R-Precision': 0.3506048489871642, 'Precision@k': 0.29512787007653535, 'Scikit-MAP': 0.0}


In [41]:
# MANUAL QUERY
def man_query(q, top_k = 10):
    print("Query-Results for:", q)
    veq_q = vectorizer.transform([q])
    scores = cosine_similarity(veq_q, tf_embeddings)[0]
    sim_indx = np.argsort(scores)[::-1].tolist()
    for i in range(top_k):
        prod_title = df_products[product_txt][sim_indx[i]]
        print(f'Product {i+1}: {prod_title}')

man_query('I want to find hiking shoes for men')

Query-Results for: I want to find hiking shoes for men
Product 1: WETIKE Girls Basketball Culture Shoes Air-Cushion Comfortable Basketball Shoes for Girls Breathable Kids Shoes Non-Slip Sneakers High Top Shoes for Boys Running Boys Tennis Shoes Size 4 Black
Product 2: I Want To Be Free
Product 3: Amazon Brand - find. Men's Boat Shoes,white (White White),US 9
Product 4: Etistta 6 Pairs of Shoes + 2 Pairs of Socks Fits for 18 inch Doll Shoes American Dolls Accessories Get Panda or Unicorn Shoes and Boots or Skates
Product 5: ALEADER hiitave Men/Womens Minimalist Barefoot Trail Running Shoes Wide Toe Glove Cross Trainers Hiking Shoes Black/Gray/Yellow US 11/11.5 Men
Product 6: DREAM PAIRS Girls Tennis Running Shoes Athletic Sports Sneakers Baby Blue Size 1 Little Kid Contact-k
Product 7: Timberland mens White Ledge Mid Waterproof Hiking Shoe, Wheat, 9.5 US
Product 8: Nike Men's Revolution 5 Running Shoe, Midnight Navy/White-Dark Obsidian, 8 Regular US
Product 9: Nike Men's Downshifter 9 R

## Sentence Transformer Untrained

In [25]:
# Or load in own sentence transformer Model
sf_model = SentenceTransformer('all-MiniLM-L6-v2').to(device)
p_embeddings = sf_model.encode(all_product_info, batch_size=64, convert_to_tensor=True).to(torch.float32).cpu()

In [26]:
class Eval_sf_undtrained(Evaluator):
    def forwards(self, query, p_emb):
        query= " ".join(i for i in query)
        q_emb = sf_model.encode(query)
        sims = sf_model.similarity(q_emb, p_emb)
        sim_score = sims.cpu().numpy()[0]        
        return sim_score

x = Eval_sf_undtrained(test_dataloader, p_embeddings)
metrics = x.evaluate()
print(metrics)
with open(f'outputs/sbert-{amazon}.txt', 'a') as f:
    f.write(f'sbert-{test_subset,training_version} :: {str(metrics)} \n\n')

{'MRR': 0.6654926356376448, 'nDCG': 0.6807798725161996, 'R-Precision': 0.42868284380223604, 'Precision@k': 0.34888930371476573, 'Scikit-MAP': 0.47476294636017496}


In [27]:
# MANUAL QUERY
def man_query(q, top_k = 10):
    print("Query-Results for:", q)
    q_emb = sf_model.encode(q)
    sims = sf_model.similarity(q_emb, p_embeddings)
    scores = sims.cpu().numpy()[0]       
    sim_indx = np.argsort(scores)[::-1].tolist()
    for i in range(top_k):
        prod_title = df_products[product_txt][sim_indx[i]]
        print(f'Product {i+1}: {prod_title}')

man_query('zero drop womens hiking shoes')

Query-Results for: zero drop womens hiking shoes
Product 1: Joomra Women Barefoot Trail Running Shoes Size 6.5-7 Ladies Wide Minimalist Zero Drop Trekking Sneakers Gym Antislip Toes Hiking Cycling Footwear Red 37
Product 2: Nike Womens Air Zoom Terra Kiger 6 Fashion Sneaker Womens Cj0220-400 Size 10
Product 3: City Classified Women's Shoes Invest Suede Peep Toe Ankle Fashion Boots,Tan,8
Product 4: CLARKS Originals Desert Boot Women's Suede Felt Chukka Shoe (9.5M) Grey
Product 5: Nike Womens Air Vapormax 360 Womens Running Casual Shoes Ck2719-400 Size 5.5
Product 6: Skechers womens Bright Sky fashion sneakers, White, 5.5 US
Product 7: Liliana Women Fashionable Peep Open Toe Cutout Chunky Heel Bootie (11, MonclairBlack)
Product 8: J. Adams Tracy Perforated Flat Bootie - Casual Open Toe Low Heel - Cut Out Shoe, Light Grey, 7.5
Product 9: Skechers Sport Women's D'lites Resilient Slip-On Mule Sneaker, Black/White, 8 M US
Product 10: ALTRA Women's AFW1953G Superior 4 Trail Running Shoes Snea

## Sentence Transformer Trained

In [28]:
sf_model_trained = SentenceTransformer('all-MiniLM-L6-v2')
sf_model_trained.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [29]:
def make_ds(dataset):
    pairs = []
    
    # Extract all queries as a list 
    query_ids = dataset['query_id'].tolist()          # query, pos_name, neg_name
    queries = dataset['query'].tolist()
    pids = dataset['product_id'].tolist()
    labels = dataset[label].tolist()

    for emb, qid, pid, l in zip(queries, query_ids, pids, labels):
        # Get all exact and rest pids
        p_row = df_products[df_products['product_id'] == pid]
        product_name = " ".join([
        str(i) for i in [
            p_row[product_infos[0]].item(),
            p_row[product_infos[1]].item(),
            p_row[product_infos[2]].item()
        ] if i and str(i).lower() != "nan"
        ])
        
        pairs.append(InputExample(texts=[emb, product_name], label = label_map[l]))
    return pairs

def collaz(batch):
    qid = [example[0] for example in batch]
    q = [example[1] for example in batch]
    return qid, q

train_dataset = make_ds(train_data)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=False)
#test_list = QueryDataset_sbert(test_data)
#testdataloader =  DataLoader(test_list, batch_size=64, shuffle=True, collate_fn = collate_sbert)

In [30]:
#train_loss = losses.CoSENTLoss(model)
#train_loss = losses.AnglELoss(model)
train_loss = losses.CosineSimilarityLoss(sf_model_trained)

In [31]:
sf_model_trained.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2, warmup_steps=100)
# save model changed epochs from 10 to 2
#sf_model_trained.save("location_for_sentence_bert_pretrained")

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


In [32]:
class Eval_sf_trained(Evaluator):
    def forwards(self, query, p_emb):
        query= " ".join(i for i in query)
        q_emb = sf_model_trained.encode(query)
        sims = sf_model_trained.similarity(q_emb, p_emb)
        sim_score = sims.cpu().numpy()[0]        
        return sim_score


p_embeddings = sf_model_trained.encode(all_product_info)

x = Eval_sf_trained(test_dataloader, p_embeddings)
metrics = x.evaluate()
print(metrics)
with open(f'outputs/sbert-trained-{amazon}.txt', 'a') as f:
    f.write(f'sbert-trained-{test_subset,training_version} :: {str(metrics)} \n\n')

{'MRR': 0.7250244520889169, 'nDCG': 0.7169340759640458, 'R-Precision': 0.47778256893916704, 'Precision@k': 0.39089042374463323, 'Scikit-MAP': 0.5277582821497365}


In [33]:
# MANUAL QUERY
def man_query(q, top_k = 10):
    print("Query-Results for:", q)
    q_emb = sf_model_trained.encode(q)
    sims = sf_model_trained.similarity(q_emb, p_embeddings)    
    scores = sims.cpu().numpy()[0]       
    sim_indx = np.argsort(scores)[::-1].tolist()
    for i in range(top_k):
        prod_title = df_products[product_txt][sim_indx[i]]
        print(f'Product {i+1}: {prod_title}')

man_query('zero drop womens hiking shoes')

Query-Results for: zero drop womens hiking shoes
Product 1: Joomra Women Barefoot Trail Running Shoes Size 6.5-7 Ladies Wide Minimalist Zero Drop Trekking Sneakers Gym Antislip Toes Hiking Cycling Footwear Red 37
Product 2: Nike Womens Air Zoom Terra Kiger 6 Fashion Sneaker Womens Cj0220-400 Size 10
Product 3: City Classified Women's Shoes Invest Suede Peep Toe Ankle Fashion Boots,Tan,8
Product 4: Liliana Women Fashionable Peep Open Toe Cutout Chunky Heel Bootie (11, MonclairBlack)
Product 5: Nike Womens Air Vapormax 360 Womens Running Casual Shoes Ck2719-400 Size 5.5
Product 6: CLARKS Originals Desert Boot Women's Suede Felt Chukka Shoe (9.5M) Grey
Product 7: Skechers Sport Women's D'Lites Memory Foam Lace-up Sneaker,White Silver,8 W US
Product 8: Skechers womens Bright Sky fashion sneakers, White, 5.5 US
Product 9: Skechers Sport Women's D'lites Resilient Slip-On Mule Sneaker, Black/White, 8 M US
Product 10: Timberland Women's Mt Hayes Waterproof Chukka Boots (7 B(M) US, Navy/Brown)


## Cross Encoder with SBERT Pretrained

In [34]:
from datasets import Dataset, Features, Value
cross_model_trained = CrossEncoder(
    "cross-encoder/ms-marco-MiniLM-L-12-v2",
    num_labels=1,
    max_length=512,    # trim/pad all pairs to this length
    device=device     # or "cpu"
)

In [35]:

train_examples = []
q, pn, l = [], [], []
cross_train_data = train_data.copy()
for _, row in cross_train_data.iterrows():
    
    query_text = f'Query: {row['query']}'
    q.append(query_text)
    
    p_row = df_products[df_products['product_id'] == row['product_id']]
    product_name = " ".join([
        str(i) for i in [
            p_row[product_infos[0]].item(),
            p_row[product_infos[1]].item(),
            p_row[product_infos[2]].item()
        ] if i and str(i).lower() != "nan"
        ])

    if 'product_color' in p_row.columns:
        product_text = f"Description: {product_name} Brand: {p_row['product_brand']} Color: {p_row['product_color']}"
    else:
        product_text = f"Description: {product_name} Category:{p_row['category hierarchy']} Class: {p_row['product_class']}" 
    
    pn.append(product_text)

    l.append(float(label_map[row[label]]))

# Just for correct type usage
features = Features({'query':  Value('string'),'product': Value('string'),'label':  Value('float32'),})

# Create Dataset
train_dataset = Dataset.from_dict(
    {
      'query':   q,
      'product': pn,
      'label':   l
    },
    features=features
)

In [36]:
p_emb = sf_model_trained.encode(all_product_info)

dev_samples = {}
for _, row in test_data.iterrows():
    query = row['query']
    qid = row['query_id']

    
    q = f'Query:{query}'
    dev_samples[qid] = {'query': q, 'positive': [], 'negative': []}

    
    q_emb = sf_model_trained.encode(query)
    sims = sf_model_trained.similarity(q_emb, p_emb)
    scores = sims.cpu().numpy()[0]        
            
    # Represents the node-indexes with decreasing similarity
    sim_indx = np.argsort(scores)[::-1].tolist()
    sim_indx = sim_indx[:100]

    # For each of the products, add them to rel or irrel
    for node_id in sim_indx:
        #print(node_id)
        pid = node_id_to_product_id.get(node_id)

        p_row = df_products[df_products['product_id'] == pid]
        #print(p_row)
        product_name = " ".join([
        str(i) for i in [
            p_row[product_infos[0]].item(),
            p_row[product_infos[1]].item(),
            p_row[product_infos[2]].item()
        ] if i and str(i).lower() != "nan"
        ])
        if 'product_color' in p_row.columns:
            product_text = f"Description: {product_name} Brand: {p_row['product_brand']} Color: {p_row['product_color']}"
        else:
            product_text = f"Description: {product_name} Category:{p_row['category hierarchy']} Class:<{p_row['product_class']}" 
        

        if label_dict.get((qid,pid), 0) == 1:
            dev_samples[qid]['positive'].append(product_text)
        else:
            dev_samples[qid]['negative'].append(product_text)
            
evaluator = CrossEncoderRerankingEvaluator(dev_samples, name='evaluate')


In [37]:
MODEL_SAVE_PATH = "model_save/croos_encoder" 
args = CrossEncoderTrainingArguments(
    # Required parameter:
    output_dir=MODEL_SAVE_PATH,
    # Optional training parameters:
    num_train_epochs=15,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True
)

In [38]:
trainer = CrossEncoderTrainer(
    model=cross_model_trained,
    args=args,
    train_dataset=train_dataset,  
    evaluator=evaluator,
)

# Start training
trainer.train()
print("Training finished.")


Token indices sequence length is longer than the specified maximum sequence length for this model (842 > 512). Running this sequence through the model will result in indexing errors


Step,Training Loss
500,0.821800
1000,0.510400
1500,0.411600
2000,0.318700


KeyboardInterrupt: 

In [ ]:
result = evaluator(cross_model_trained)  
print(result)

In [ ]:
with open(f'cross + sbert.txt', 'a') as f:
    f.write(f'cross+sbert-{test_subset,training_version, amazon} :: {str(result)} \n\n')

In [ ]:
# MANUAL QUERY
def man_query(q, top_k = 10):
    print("Query-Results for:", q)
    q_emb = sf_model_trained.encode(q)
    sims = sf_model_trained.similarity(q_emb, p_embeddings)    
    scores = sims.cpu().numpy()[0]       
    sim_indx = np.argsort(scores)[::-1].tolist()[:100]
    
    for node_id in sim_indx:
        #print(node_id)
        pid = node_id_to_product_id.get(node_id)

        p_row = df_products[df_products['product_id'] == pid]
        #print(p_row)
        product_name = " ".join([
        str(i) for i in [
            p_row[product_infos[0]].item(),
            p_row[product_infos[1]].item(),
            p_row[product_infos[2]].item()
        ] if i and str(i).lower() != "nan"
        ])
        if 'product_color' in p_row.columns:
            product_text = f"Description: {product_name} Brand: {p_row['product_brand']} Color: {p_row['product_color']}"
        else:
            product_text = f"Description: {product_name} Category:{p_row['category hierarchy']} Class:<{p_row['product_class']}" 

    

        
    for i in range(top_k):
        prod_title = df_products['product_title'][sim_indx[i]]
        print(f'Product {i+1}: {prod_title}')

man_query('zero drop womens hiking shoes')